# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.3 Epdemics

#### Exercise 2.9: Simulate an epidemic outbreak

In [31]:
# imports and load data
import json
import networkx as nx
import epidemics_helper

with open("../data/nyc_augmented_network.json") as file:
    network_data = json.load(file)


In [32]:
G = nx.Graph()
# network_data['nodes'] # list of dicts
def form_nodes(dict):
    return (dict['id'],{'coordinates': dict['coordinates']})
nodes = map(form_nodes ,network_data['nodes'])

G.add_nodes_from(nodes)

def form_edges(dict):
    return (dict['source'],dict['target'])
edges = map(form_edges ,network_data['links'])

G.add_edges_from(edges)


In [33]:
G.nodes[0]['coordinates']

[-73887234, 40714842]

In [34]:
# set all nodes into state Susceptible
nx.set_node_attributes(G,'S', name= 'state')


In [35]:
# 1. The first infection is at node 23654

In [36]:
# simulate epidemic over 100 days
# 1. The first infection is at node 23654
sir = epidemics_helper.SimulationSIR(G, beta=10.0, gamma=0.1)
sir.launch_epidemic(source=23654, max_time=100.0)

Epidemic stopped after 106.01 days | 106.01 days elapsed | 0.1% susceptible, 0.0% infected, 99.9% recovered


In [ ]:
# plot percentage of s,i,r over time

inf_time_nodes = {}
rec_time_nodes = {}
for node in G.nodes():
    inf_time_nodes[node] = sir.inf_time[node]
    rec_time_nodes[node] = sir.rec_time[node]

In [ ]:
# how long until 60% is infected

In [ ]:
# use coordinates to visualize graph

---

### 2.3.1 Stop the apocalypse!

#### Exercise 2.10: Strategy 1

#### Exercise 2.11: Strategy 2